# How to use st_hsdatalog HSD_utils package - \[Data format conversion Features\]
---
<br>
<p>import st_hsdatalog.HSDatalog and HSDatalogConverter modules</p>

In [ ]:
from st_hsdatalog.HSD_utils.converters import HSDatalogConverter
from st_hsdatalog.HSD.HSDatalog import HSDatalog

<span style="font-style: italic; color:#909090;"><span style="color:cyan;">*[Notebook utils] </span> -> Update this boolean to select the right example folder and sensor names in next code cells</span>

In [ ]:
hsd_2 = True

HSDatalog will be only used to extract Pandas dataframes in a simple way (extracting them from an HSDatalog acquisition folder), as shown in the [nb_hsdatalog_core.ipynb] notebook.

In [ ]:
if hsd_2:
    acq_folder = "..\\..\\STWIN.box_acquisition_examples\\20221017_13_04_39"
    sensor_name = "iis3dwb_acc"
else:
    acq_folder = "..\\..\\STWIN_acquisition_examples\\STWIN_00002"
    sensor_name = "IIS3DWB"

hsd_factory = HSDatalog()
hsd= hsd_factory.create_hsd(acq_folder)

sensor = HSDatalog.get_sensor(hsd, sensor_name)
dataframe = HSDatalog.get_dataframe(hsd, sensor)
print(dataframe)

# Data Conversion
---
<p>The HSDatalogConverter module works independently from the core module, as long as the data to be converted are Pandas DataFrames.</p>
Starting from a DataFrame it is possible to convert the data of an acquisition in the following formats:
<ul>
<li> CSV, TSV
<li> Cartesiam format
<li> ST UNICO format
<li> WAV (Only for Microphones)
</ul>
To access the conversion features, we have to:
<ul>
<li> Import the HSD_utils.converters module (already done in the first code cell).
<li> choose a folder to save the conversion output files
</ul>


In [ ]:
from st_hsdatalog.HSD_utils.converters import HSDatalogConverter
import os

if hsd_2:
    output_folder = ".\\STWIN.box_Exported_data_folder"
else:
    output_folder = ".\\STWIN_Exported_data_folder"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

## CSV, TSV Conversion

In [ ]:
if hsd_2:
    sensor_name = "stts22h_temp"
else:
    sensor_name = "HTS221"

sensor = HSDatalog.get_sensor(hsd, sensor_name)
dataframe = HSDatalog.get_dataframe(hsd, sensor)

exported_sensor_path = os.path.join(output_folder,sensor_name)

print("CSV Conversion Started")
[HSDatalogConverter.to_csv(x, exported_sensor_path) for x in dataframe]
print("CSV Conversion Ended")
print("TSV Conversion Started")
[HSDatalogConverter.to_tsv(x, exported_sensor_path) for x in dataframe]
print("TSV Conversion Ended")

## Nanoedge format Conversion

In [ ]:
if hsd_2:
    sensor_name = "iis3dwb_acc"
else:
    sensor_name = "IIS3DWB"

dataframe = HSDatalog.get_dataframe(hsd, sensor)
for df in dataframe:
    HSDatalogConverter.to_nanoedge_format(output_folder, sensor_name, df, signal_length = 100, signal_increment = 75)

## ST UNICO format Conversion
Available outpuf formats currently accepted in input by Unico GUI: txt (default), csv, tsv. 
- Unlabeled acquisition. Even if present, the tag data in the acquisition is not used

In [ ]:
if hsd_2:
    sensor_name = "ism330dhcx_gyro"
else:
    sensor_name = "ISM330DHCX"
    
dataframe = HSDatalog.get_dataframe(hsd, sensor)
HSDatalogConverter.to_unico(output_folder, sensor_name, dataframe, out_format = "csv")

- Labeled Acquisition. If available, the tag data are read and used to split the converted files

In [ ]:
if hsd_2:
    sensor_name = "ism330dhcx_gyro"
else:
    sensor_name = "ISM330DHCX"

dataframe = HSDatalog.get_dataframe(hsd, sensor, labeled=True)
data_tags = HSDatalog.get_data_stream_tags(hsd, sensor)
print(data_tags)

HSDatalogConverter.to_unico(output_folder, sensor_name, dataframe, data_tags, "csv")

## WAV Conversion

<span style="font-style: italic; color:#909090;">Conversion process is done in "batch mode", so this feature could be used even with large files as input.</span>
- example with IMP23ABSU Analog microphone

In [ ]:
import numpy as np
from IPython.display import Audio

if hsd_2:
    acq_folder = "..\\..\\STWIN.box_acquisition_examples\\20221017_12_50_55"
    sensor_name = "imp23absu_mic"
else:
    acq_folder = "..\\..\\STWIN_acquisition_examples\\STWIN_20210205_15_47_16"
    sensor_name = "IMP23ABSU"

hsd= hsd_factory.create_hsd(acq_folder)
    
sensor = HSDatalog.get_sensor(hsd, sensor_name)
sensor_odr = HSDatalog.get_sensor_odr(hsd, sensor)
print("ODR: {}".format(sensor_odr))

wav_file_name = os.path.join(output_folder, "{}.wav".format(sensor_name))
audio_data = HSDatalog.get_data_and_timestamps(hsd, sensor)[0][0]
pcm_data = audio_data.astype(np.int16)
HSDatalogConverter.to_wav(pcm_data,wav_file_name, sensor_odr)
Audio(wav_file_name)

- example with IMP34DT05 Digital microphone

In [ ]:
import numpy as np
from IPython.display import Audio

if hsd_2:
    sensor_name = "imp34dt05_mic"
else:
    sensor_name = "IMP34DT05"

sensor = HSDatalog.get_sensor(hsd, sensor_name)
sensor_odr = HSDatalog.get_sensor_odr(hsd, sensor)
print("ODR: {}".format(sensor_odr))

wav_file_name = os.path.join(output_folder, "{}.wav".format(sensor_name))
audio_data = HSDatalog.get_data_and_timestamps(hsd, sensor)[0][0]
pcm_data = audio_data.astype(np.int16)
HSDatalogConverter.to_wav(pcm_data,wav_file_name, sensor_odr)
Audio(wav_file_name)